<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Chapter 4

In [2]:
import pandas as pd
df = pd.read_csv('enron_emails_clean.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 6 columns):
Message-ID       2090 non-null object
From             2090 non-null object
To               2090 non-null object
Date             2090 non-null object
content          2090 non-null object
clean_content    2086 non-null object
dtypes: object(6)
memory usage: 98.0+ KB


In [7]:
# Find all cleaned emails that contain 'sell enron stock'
mask = df['clean_content'].str.contains('sell enron stock', na=False)

# Select the data from df using the mask
print(df.loc[mask])

                                       Message-ID                        From  \
154  <6336501.1075841154311.JavaMail.evans@thyme>  ('sarah.palmer@enron.com')   

                             To                 Date  \
154  ('sarah.palmer@enron.com')  2002-02-01 14:53:35   

                                               content  \
154  \nJoint Venture: A 1997 Enron Meeting Belies O...   

                                         clean_content  
154  joint venture enron meeting belies officers cl...  


In [10]:
# Create a list of terms to search for
searchfor = ['enron stock', 'sell stock', 'stock bonus', 'sell enron stock']

# Filter cleaned emails on searchfor list and select from df 
filtered_emails = (df.loc[df['clean_content'].str.contains('|'.join(searchfor), na=False)])
print(filtered_emails)

                                         Message-ID  \
0      <8345058.1075840404046.JavaMail.evans@thyme>   
1      <1512159.1075863666797.JavaMail.evans@thyme>   
2     <26118676.1075862176383.JavaMail.evans@thyme>   
3     <10369289.1075860831062.JavaMail.evans@thyme>   
4     <26728895.1075860815046.JavaMail.evans@thyme>   
5     <24716667.1075860824904.JavaMail.evans@thyme>   
6     <33107398.1075861369659.JavaMail.evans@thyme>   
7     <26091852.1075860822907.JavaMail.evans@thyme>   
8     <19319259.1075862176360.JavaMail.evans@thyme>   
9      <6656919.1075860824395.JavaMail.evans@thyme>   
10    <18830843.1075860818403.JavaMail.evans@thyme>   
11    <28795989.1075852092444.JavaMail.evans@thyme>   
12    <17859589.1075862385936.JavaMail.evans@thyme>   
13     <8415659.1075860990190.JavaMail.evans@thyme>   
14    <15501472.1075861214988.JavaMail.evans@thyme>   
15    <15184740.1075860835623.JavaMail.evans@thyme>   
16     <1678648.1075842053524.JavaMail.evans@thyme>   
17    <307

In [12]:
# Create flag variable where the emails match the searchfor terms
df['flag'] = np.where((df['clean_content'].str.contains('|'.join(searchfor)) == True), 1, 0)

# Count the values of the flag variable
count = df['flag'].value_counts()
print(count)

0    1776
1     314
Name: flag, dtype: int64


In [17]:
# Import nltk packages and string 
from nltk.corpus import stopwords
import string

# Define stopwords to exclude
stop = set(stopwords.words('english'))
stop.update(("to","cc","subject","http","from","sent", "ect", "u", "fwd", "www", "com"))

# Define punctuations to exclude and lemmatizer
exclude = set(string.punctuation)

In [25]:
# Import the lemmatizer from nltk
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

# Define word cleaning function
def clean(text, stop):
    text = text.rstrip()
	# Remove stopwords
    stop_free = " ".join([word for word in text.lower().split() if ((word not in stop) and (not word.isdigit()))])
	# Remove punctuations
    punc_free = ''.join(word for word in stop_free if word not in exclude)
	# Lemmatize all words
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())      
    return normalized

# Clean the emails in df and print results
text_clean=[]
for text in df['clean_content'].dropna():
    text_clean.append(clean(text, stop).split())    
print(text_clean[:10])

[['investools', 'advisory', 'free', 'digest', 'trusted', 'investment', 'advice', 'unsubscribe', 'free', 'newsletter', 'please', 'see', 'issue', 'fried', 'sell', 'stock', 'gain', 'month', 'km', 'rowe', 'january', 'index', 'confirms', 'bull', 'market', 'aloy', 'small', 'cap', 'advisor', 'earns', 'lbix', 'compounding', 'return', 'pine', 'tree', 'pcl', 'undervalued', 'high', 'yield', 'bank', 'put', 'customer', 'first', 'aso', 'word', 'sponsor', 'top', 'wall', 'street', 'watcher', 'ben', 'zacks', 'year', 'year', 'gain', 'moving', 'best', 'brightest', 'wall', 'street', 'big', 'money', 'machine', 'earned', 'ben', 'zacks', 'five', 'year', 'average', 'annual', 'gain', 'start', 'outperforming', 'long', 'term', 'get', 'zacks', 'latest', 'stock', 'buylist', 'free', 'day', 'trial', 'investools', 'c', 'go', 'zaks', 'mtxtu', 'zakstb', 'investools', 'advisory', 'john', 'brobst', 'investools', 'fried', 'sell', 'stock', 'lock', 'month', 'km', 'david', 'fried', 'know', 'stock', 'undervalued', 'company', 

In [28]:
# Import the packages
import gensim
from gensim import corpora

# Define the dictionary
dictionary = corpora.Dictionary(text_clean)

# Define the corpus 
corpus = [dictionary.doc2bow(text) for text in text_clean]

# Print corpus and dictionary
print(corpus)
print(dictionary)

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 6), (6, 1), (7, 2), (8, 4), (9, 1), (10, 1), (11, 3), (12, 2), (13, 1), (14, 5), (15, 3), (16, 1), (17, 3), (18, 1), (19, 1), (20, 1), (21, 5), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 3), (32, 1), (33, 3), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 4), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 4), (56, 1), (57, 4), (58, 9), (59, 5), (60, 1), (61, 8), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 12), (82, 2), (83, 2), (84, 1), (85, 1), (86, 3), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 4), (96, 1), (97, 1), (98, 2), (99, 2), (100, 1), (101, 2), (102, 5), (103, 1), (104, 3), (105, 8), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1

In [29]:
# Define the LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=5)

# Save the topics and top 5 words
topics = ldamodel.print_topics(num_words=5)

# Print the results
for topic in topics:
    print(topic)

(0, '0.030*"image" + 0.009*"se" + 0.008*"ne" + 0.008*"sp" + 0.008*"wscc"')
(1, '0.048*"td" + 0.035*"net" + 0.033*"money" + 0.031*"tr" + 0.028*"width"')
(2, '0.054*"enron" + 0.013*"hou" + 0.011*"message" + 0.011*"pm" + 0.010*"original"')
(3, '0.018*"enron" + 0.009*"company" + 0.005*"employee" + 0.005*"stock" + 0.005*"energy"')
(4, '0.011*"e" + 0.009*"mail" + 0.009*"image" + 0.006*"go" + 0.005*"message"')


In [30]:
def get_topic_details(ldamodel, corpus):
    topic_details_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topic_details_df = topic_details_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
    topic_details_df.columns = ['Dominant_Topic', '% Score', 'Topic_Keywords']
    return topic_details_df

In [31]:
# Run get_topic_details function and check the results
print(get_topic_details(ldamodel, corpus))

      Dominant_Topic  % Score  \
0                3.0   0.6055   
1                3.0   0.6914   
2                4.0   0.5447   
3                3.0   0.9935   
4                3.0   0.9934   
5                3.0   0.9934   
6                3.0   0.8654   
7                3.0   0.9934   
8                4.0   0.5597   
9                3.0   0.9934   
10               3.0   0.9935   
11               3.0   0.7137   
12               3.0   0.9945   
13               3.0   0.9835   
14               3.0   0.9835   
15               3.0   0.9933   
16               3.0   0.8770   
17               2.0   0.5109   
18               3.0   0.9975   
19               3.0   0.9740   
20               3.0   0.9933   
21               3.0   0.9941   
22               3.0   0.9933   
23               3.0   0.9934   
24               3.0   0.8547   
25               3.0   0.9695   
26               3.0   0.9931   
27               3.0   0.5722   
28               3.0   0.6747   
29        

In [32]:
# Add original text to topic details in a dataframe
contents = pd.DataFrame({'Original text': text_clean})
topic_details = pd.concat([get_topic_details(ldamodel, corpus), contents], axis='columns')
topic_details.head()

,Dominant_Topic,% Score,Topic_Keywords,Original text
0,3.0,0.6055,"enron, company, employee, stock, energy, new, ...","[investools, advisory, free, digest, trusted, ..."
1,3.0,0.6914,"enron, company, employee, stock, energy, new, ...","[forwarded, richard, b, sander, hou, pm, justi..."
2,4.0,0.5447,"e, mail, image, go, message, email, team, clic...","[hey, wearing, target, purple, shirt, today, m..."
3,3.0,0.9935,"enron, company, employee, stock, energy, new, ...","[leslie, milosevich, santa, clara, avenue, ala..."
4,3.0,0.9934,"enron, company, employee, stock, energy, new, ...","[rini, twait, e, th, ave, longmont, co, rtwait..."


In [34]:
# Create flag for text highest associated with topic 3
topic_details['flag'] = np.where((topic_details['Dominant_Topic'] == 3.0), 1, 0)
print(topic_details.head())


   Dominant_Topic  % Score                                     Topic_Keywords  \
0             3.0   0.6055  enron, company, employee, stock, energy, new, ...   
1             3.0   0.6914  enron, company, employee, stock, energy, new, ...   
2             4.0   0.5468  e, mail, image, go, message, email, team, clic...   
3             3.0   0.9935  enron, company, employee, stock, energy, new, ...   
4             3.0   0.9934  enron, company, employee, stock, energy, new, ...   

                                       Original text  flag  
0  [investools, advisory, free, digest, trusted, ...     1  
1  [forwarded, richard, b, sander, hou, pm, justi...     1  
2  [hey, wearing, target, purple, shirt, today, m...     0  
3  [leslie, milosevich, santa, clara, avenue, ala...     1  
4  [rini, twait, e, th, ave, longmont, co, rtwait...     1  
